# 🤟 VideoMAE WLASL Training - Sistema Completo

**Autor:** Rafael Ovalle - Tesis UNAB  
**Dataset:** WLASL100/WLASL300 (V1 y V2)  
**Modelo:** VideoMAE (MCG-NJU/videomae-base-finetuned-kinetics)  

---

## 📋 Estructura del Notebook

1. ✅ **Configuración Inicial** - Verificar GPU y entorno
2. 📦 **Clonar Repositorio y Dataset** - Desde GitHub y Google Drive
3. 🔍 **Verificar Dataset** - Estructura y splits
4. ⚙️ **Configurar Hiperparámetros** - Elegir dataset y versión
5. 🚀 **Entrenamiento** - Largo o corto de prueba
6. 📊 **Evaluación** - Listar modelos y evaluar
7. 📈 **Visualización** - Métricas y gráficos
8. 💾 **Descargar Resultados** - Comprimir y exportar
9. 🛠️ **Utilidades** - Monitorear GPU y limpiar memoria

---

**IMPORTANTE:**  
- Este notebook está diseñado para **Google Colab conectado a VM local**
- Usa `!python3` para comandos en la VM
- Dataset clonado con `shutil` desde Google Drive
- Ruta dataset: `/home/ov4lle/AtiendeSenas-MVP/data/`


---
# 1️⃣ Configuración Inicial


## 1.1 Verificar GPU


In [ ]:
# Verificar GPU con nvidia-smi
!nvidia-smi

In [ ]:
# Verificar GPU en PyTorch
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ GPU no disponible - entrenamiento será en CPU (muy lento)")

## 1.2 Verificar Uso de Disco


In [ ]:
# Verificar espacio en disco
!df -h | grep -E "Filesystem|/home"

---
# 2️⃣ Clonar Repositorio y Dataset


## 2.1 Clonar Repositorio desde GitHub


In [ ]:
# Clonar repositorio (solo si no existe)
import os

REPO_PATH = "/home/ov4lle/AtiendeSenas-MVP"

if not os.path.exists(REPO_PATH):
    print("📦 Clonando repositorio desde GitHub...")
    !git clone https://github.com/Ov4llezz/AtiendeSenas-MVP.git {REPO_PATH}
else:
    print("✅ Repositorio ya existe")
    print("🔄 Actualizando desde GitHub...")
    !cd {REPO_PATH} && git pull

## 2.2 Montar Google Drive


In [ ]:
# Montar Google Drive (Colab)
from google.colab import drive
drive.mount('/content/drive')

## 2.3 Clonar Dataset desde Google Drive


In [ ]:
# Configurar rutas
import shutil
from pathlib import Path

# Ruta en Google Drive (AJUSTAR según tu estructura)
DRIVE_DATASET_PATH = "/content/drive/MyDrive/TESIS/datasets"  # CAMBIAR ESTO

# Ruta destino en VM
VM_DATA_PATH = "/home/ov4lle/AtiendeSenas-MVP/data"

# Crear directorio si no existe
os.makedirs(VM_DATA_PATH, exist_ok=True)

print(f"🔍 Buscando datasets en: {DRIVE_DATASET_PATH}")
print(f"📂 Destino: {VM_DATA_PATH}")

In [ ]:
# Lista de datasets a copiar
DATASETS = ["wlasl100", "wlasl300", "wlasl100_v2", "wlasl300_v2"]

for dataset_name in DATASETS:
    src = os.path.join(DRIVE_DATASET_PATH, dataset_name)
    dst = os.path.join(VM_DATA_PATH, dataset_name)
    
    if os.path.exists(src):
        if not os.path.exists(dst):
            print(f"\n📦 Copiando {dataset_name}...")
            shutil.copytree(src, dst)
            print(f"✅ {dataset_name} copiado")
        else:
            print(f"✅ {dataset_name} ya existe en VM")
    else:
        print(f"⚠️ {dataset_name} no encontrado en Drive")

print("\n✅ Proceso de copia finalizado")

---
# 3️⃣ Verificar Dataset


In [ ]:
# Cambiar al directorio del proyecto
%cd /home/ov4lle/AtiendeSenas-MVP

In [ ]:
# Verificar estructura de los 4 datasets
from colab_utils.dataset_utils import verify_all_datasets

verify_all_datasets(data_root="/home/ov4lle/AtiendeSenas-MVP/data")

---
# 4️⃣ Configurar Hiperparámetros y Elegir Dataset


## 4.1 Configuración de Hiperparámetros

**Parámetros disponibles:**
- `dataset_type`: "wlasl100" o "wlasl300"
- `version`: "v1" (train/val/test separados) o "v2" (train+val combinados)
- `batch_size`: Tamaño del batch (default: 6)
- `max_epochs`: Número máximo de epochs (default: 30)
- `learning_rate`: Learning rate inicial (default: 1e-5)
- `patience`: Epochs sin mejora para early stopping (default: 10)
- `weight_decay`: Regularización L2 (default: 0.0)
- `label_smoothing`: Label smoothing (default: 0.0)
- `class_weighted`: Usar pesos de clases (default: False)
- `freeze_backbone`: Congelar backbone VideoMAE (default: False)


In [ ]:
from colab_utils.config import create_config, print_config

# ============================================================
#   CONFIGURACIÓN DEL EXPERIMENTO
# ============================================================

config = create_config(
    # Dataset y versión
    dataset_type="wlasl100",      # "wlasl100" o "wlasl300"
    version="v2",                  # "v1" o "v2"
    data_root="/home/ov4lle/AtiendeSenas-MVP/data",
    
    # Hiperparámetros principales
    batch_size=6,
    max_epochs=30,
    learning_rate=1e-5,
    patience=10,
    
    # Regularización
    weight_decay=0.0,
    label_smoothing=0.0,
    class_weighted=False,
    freeze_backbone=False,         # False = entrenar todas las capas
    
    # Otros
    warmup_ratio=0.1,
    min_lr=1e-6,
    gradient_clip=1.0,
    num_workers=2,
    save_every=5
)

# Mostrar configuración
print_config(config)

---
# 5️⃣ Entrenamiento


## 5.1 Entrenamiento CORTO (Prueba - 3 epochs)


In [ ]:
# Entrenamiento corto de prueba (3 epochs)
from colab_utils.training import train_model
from colab_utils.dataset import create_dataloaders

# Crear dataloaders
train_loader, val_loader, train_dataset = create_dataloaders(config)

# Configuración temporal para prueba rápida
config_prueba = config.copy()
config_prueba['max_epochs'] = 3
config_prueba['save_every'] = 1

print("\n🚀 Iniciando entrenamiento de PRUEBA (3 epochs)...\n")

# Entrenar
model, history, checkpoint_dir, log_dir = train_model(
    config_prueba,
    train_loader,
    val_loader,
    train_dataset
)

print(f"\n✅ Entrenamiento de prueba completado")
print(f"📁 Checkpoints: {checkpoint_dir}")
print(f"📊 Logs: {log_dir}")

## 5.2 Entrenamiento COMPLETO (30 epochs)


In [ ]:
# Entrenamiento completo (30 epochs o lo configurado)
from colab_utils.training import train_model
from colab_utils.dataset import create_dataloaders

# Crear dataloaders
train_loader, val_loader, train_dataset = create_dataloaders(config)

print(f"\n🚀 Iniciando entrenamiento COMPLETO ({config['max_epochs']} epochs)...\n")

# Entrenar
model, history, checkpoint_dir, log_dir = train_model(
    config,
    train_loader,
    val_loader,
    train_dataset
)

print(f"\n✅ Entrenamiento completado")
print(f"📁 Checkpoints: {checkpoint_dir}")
print(f"📊 Logs: {log_dir}")

---
# 6️⃣ Evaluación de Modelo


## 6.1 Listar Modelos Disponibles


In [ ]:
# Listar modelos V1 (wlasl100/300 originales)
!python3 scripts/test.py --list

In [ ]:
# Listar modelos V2 (wlasl100_v2/300_v2 - train+val combinados)
!python3 scripts_v2/test.py --list

## 6.2 Evaluar Modelo por ID


In [ ]:
# Evaluar modelo V1 por ID
# Ejemplo: si el ID es 1, evalúa el run #1

MODEL_ID_V1 = 1  # CAMBIAR según el listado

!python3 scripts/test.py --run-id {MODEL_ID_V1}

In [ ]:
# Evaluar modelo V2 por ID
MODEL_ID_V2 = 1  # CAMBIAR según el listado

!python3 scripts_v2/test.py --run-id {MODEL_ID_V2}

## 6.3 Evaluar Modelo por Path Específico


In [ ]:
# Evaluar modelo V1 por path específico
CHECKPOINT_PATH_V1 = "models/checkpoints/run_20251130_123456/best_model.pt"  # CAMBIAR

!python3 scripts/test.py --checkpoint {CHECKPOINT_PATH_V1}

In [ ]:
# Evaluar modelo V2 por path específico
CHECKPOINT_PATH_V2 = "models_v2/checkpoints/run_20251130_123456/best_model.pt"  # CAMBIAR

!python3 scripts_v2/test.py --checkpoint {CHECKPOINT_PATH_V2}

---
# 7️⃣ Visualización de Resultados


## 7.1 Matriz de Confusión


In [ ]:
from IPython.display import Image, display
import glob

# Buscar la matriz de confusión más reciente
confusion_matrices = sorted(glob.glob("evaluation_results*/confusion_matrix.png"), reverse=True)

if confusion_matrices:
    print(f"📊 Mostrando: {confusion_matrices[0]}")
    display(Image(filename=confusion_matrices[0]))
else:
    print("⚠️ No se encontraron matrices de confusión. Ejecuta la evaluación primero.")

## 7.2 Métricas Detalladas


In [ ]:
import json

# Buscar el reporte JSON más reciente
json_reports = sorted(glob.glob("evaluation_results*/test_results_*.json"), reverse=True)

if json_reports:
    with open(json_reports[0], 'r') as f:
        results = json.load(f)
    
    print("="*70)
    print(f"{'RESULTADOS DE EVALUACIÓN':^70}")
    print("="*70)
    print(f"Accuracy Total: {results['overall_accuracy']:.2f}%")
    print(f"Top-3 Accuracy: {results.get('top3_accuracy', 'N/A')}")
    print(f"Top-5 Accuracy: {results.get('top5_accuracy', 'N/A')}")
    print(f"\nPrecision (macro): {results.get('precision_macro', 'N/A')}")
    print(f"Recall (macro): {results.get('recall_macro', 'N/A')}")
    print(f"F1-score (macro): {results.get('f1_macro', 'N/A')}")
    print("="*70)
else:
    print("⚠️ No se encontraron reportes JSON. Ejecuta la evaluación primero.")

## 7.3 Curvas de Entrenamiento (TensorBoard)


In [ ]:
# Cargar TensorBoard
%load_ext tensorboard

# Mostrar logs del último run
# CAMBIAR según el directorio del último run
LOG_DIR = "runs/v2/wlasl100_v2"  # o "runs_v2/..."

%tensorboard --logdir {LOG_DIR}

---
# 8️⃣ Descargar Resultados


## 8.1 Comprimir y Descargar Todo


In [ ]:
import shutil
from datetime import datetime

# Crear archivo zip con todos los resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_zip = f"resultados_completos_{timestamp}"

# Agregar al zip: checkpoints, logs, evaluation_results
!zip -r {output_zip}.zip models/ models_v2/ runs/ runs_v2/ evaluation_results/ evaluation_results_v2/

print(f"\n✅ Archivo creado: {output_zip}.zip")
print(f"📦 Tamaño: ", end="")
!du -h {output_zip}.zip

In [ ]:
# Descargar archivo zip a tu máquina local
from google.colab import files

files.download(f"{output_zip}.zip")

## 8.2 Descargar Archivos Individuales


In [ ]:
# Descargar matriz de confusión
if confusion_matrices:
    files.download(confusion_matrices[0])

# Descargar reporte JSON
if json_reports:
    files.download(json_reports[0])

# Descargar mejor modelo
best_models = sorted(glob.glob("models*/checkpoints/*/best_model.pt"), reverse=True)
if best_models:
    files.download(best_models[0])

## 8.3 Copiar a Google Drive


In [ ]:
# Copiar resultados a Google Drive
DRIVE_OUTPUT_PATH = "/content/drive/MyDrive/TESIS/resultados"  # CAMBIAR ESTO

os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)

# Copiar zip
shutil.copy(f"{output_zip}.zip", DRIVE_OUTPUT_PATH)

print(f"✅ Resultados copiados a: {DRIVE_OUTPUT_PATH}")

---
# 9️⃣ Utilidades


## 9.1 Monitorear GPU en Tiempo Real


In [ ]:
# Monitorear GPU cada 2 segundos (CTRL+C para detener)
!watch -n 2 nvidia-smi

## 9.2 Limpiar Memoria GPU


In [ ]:
# Limpiar memoria GPU
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

print("✅ Memoria GPU limpiada")
print(f"Memoria GPU usada: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Memoria GPU reservada: {torch.cuda.memory_reserved() / 1e9:.2f} GB")

## 9.3 Ver Procesos en GPU


In [ ]:
# Ver qué procesos están usando la GPU
!nvidia-smi --query-compute-apps=pid,process_name,used_memory --format=csv

---
# ✅ Notebook Completado

**Autor:** Rafael Ovalle - Tesis UNAB  
**Versión:** Final  
**Fecha:** Noviembre 2025  

---

## 📌 Notas Finales

- Este notebook está optimizado para **Google Colab + VM local**
- Usa `!python3` para todos los comandos
- Los datasets V2 ya tienen `train = train + val`
- Modelos V1 y V2 están separados en directorios distintos
- Recuerda ajustar las rutas de Google Drive según tu estructura

---

## 🚀 Próximos Pasos

1. Entrenar con configuraciones óptimas
2. Evaluar en test set
3. Analizar matriz de confusión y errores
4. Iterar con ajustes de hiperparámetros
5. Documentar resultados para la tesis

---

**¡Buena suerte con el entrenamiento! 🤟**
